In [5]:
import boto3
import requests

## Getting access token

run in terminal

```bash
source env.sh
python lakey_service/manage.py create_account --type ADMIN <here_put_you_email>
python lakey_service/manage.py create_auth_token <here_put_you_email>
```

In [8]:
access_token = (
    'eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpZCI6MiwiZW1haWwiOiJtYWNpZWpAY29zcGh'
    'lcmUub3JnIiwidHlwZSI6IkFETUlOIiwiZXhwIjoxNTU2MzEzMjI1fQ.ncutBBeSPMjWDX6BDAdU'
    'CXmcRSYFRsysPAySt0w79WI')

## Create Catalogue Item

In [19]:
response = requests.post(
    'http://localhost:8887/catalogue/items/',
    json={
        'name': 'lakey.shopping',
        'maintained_by_id': 1,
        'executor_type': 'ATHENA',        
        'spec': [
            {
                'name': 'email',
                'type': 'STRING',
                'size': None,
                'is_nullable': False,
                'distribution': None,
            },
            {
                'name': 'name',
                'type': 'STRING',
                'size': None,
                'is_nullable': False,
                'distribution': None,
            },
            {
                'name': 'price',
                'type': 'FLOAT',
                'size': None,
                'is_nullable': False,
                'distribution': None,
            }, 
            {
                'name': 'quantity',
                'type': 'INTEGER',
                'size': None,
                'is_nullable': False,
                'distribution': None,
            },             
        ]
    },
    headers={'Authorization': f'Bearer {access_token}'})

response

In [21]:
response = requests.get(
    'http://localhost:8887/catalogue/items/',
    headers={'Authorization': f'Bearer {access_token}'})

response.json()

{'items': [{'id': 1,
   'name': 'lakey.shopping',
   'spec': [{'name': 'email',
     'size': None,
     'type': 'STRING',
     'is_nullable': False,
     'distribution': None},
    {'name': 'name',
     'size': None,
     'type': 'STRING',
     'is_nullable': False,
     'distribution': None},
    {'name': 'price',
     'size': None,
     'type': 'FLOAT',
     'is_nullable': False,
     'distribution': None},
    {'name': 'quantity',
     'size': None,
     'type': 'INTEGER',
     'is_nullable': False,
     'distribution': None}],
   'sample': [],
   'executor_type': 'ATHENA',
   'created_by': {'id': 2,
    'email': 'maciej@cosphere.org',
    'type': 'ADMIN',
    '@type': 'account'},
   'updated_by': {'id': 2,
    'email': 'maciej@cosphere.org',
    'type': 'ADMIN',
    '@type': 'account'},
   'maintained_by': {'id': 1,
    'email': 'sbcm@viessmann.com',
    'type': 'ADMIN',
    '@type': 'account'},
   '@type': 'catalogue_item'}],
 '@type': 'catalogue_items_list',
 '@event': 'CATALOGUE

## Create Download Request

## Download data locally (this should happend in Lakey UI)

In [ ]:
requests.get()
